# Hands-on Exercise: Processing Legacy Lab Data for ML

## Duration: 90 minutes

### Overview

In this exercise, you'll learn how to: 
1. Clean and standardize heterogeneous lab data 
2. Merge data from multiple sources
3. Handle missing values and inconsistencies
4. Prepare data for ML model training
5. Build a basic transformer for antibody property prediction


### Feature Engineering and ML Preparation

You should start by cleaning the data using the `antibody_data_preprocessing` notebook.

# Transformer architecture

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from typing import List, Tuple

class AntibodyDataset(Dataset):
    def __init__(self, merged_experimental_df: pd.DataFrame):
        self.data = pd.read_csv(merged_experimental_df)
        
        # Create amino acid vocabulary
        self.aa_vocab = {aa: idx for idx, aa in enumerate('ACDEFGHIKLMNPQRSTVWY')}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Convert sequences to numerical arrays
        vh_tensor = torch.tensor([self.aa_vocab[aa] for aa in row['sequences_hc_sequence']], dtype=torch.long)
        vl_tensor = torch.tensor([self.aa_vocab[aa] for aa in row['sequences_lc_sequence']], dtype=torch.long)

        # Extract target variables
        target_columns = ['binding_affinity_kd', 'thermostability_tm1_celsius', 'asec_monomerpct']
        targets = row.loc[target_columns].astype(float).values

        # Pad the VL to have the same length as the VH
        return pad_sequence([vh_tensor, vl_tensor]), torch.tensor(targets)

class AntibodyTransformer(nn.Module):
    def __init__(self, vocab_size=20, d_model=128, nhead=8, num_layers=3, dropout=0.1):
        super().__init__()

        # Embedding layer for amino acid sequences
        self.embedding = nn.Embedding(vocab_size, d_model)

        # Positional encoding
        self.pos_encoder = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=512,
            dropout=dropout
        )

        # Transformer encoder
        self.transformer = nn.TransformerEncoder(
            encoder_layer=self.pos_encoder,
            num_layers=num_layers
        )

        # Output layers
        self.fc1 = nn.Linear(d_model * 2, 256)  # *2 because we have VH and VL
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)  # 3 outputs: aggregation, KD, Tm1

        self.dropout = nn.Dropout(dropout)

        self.double()

    def forward(self, x):
        # x shape: [batch_size, 2, seq_length]
        batch_size = x.shape[0]

        # Process VH and VL sequences separately
        vh = x[:, 0, :]  # [batch_size, seq_length]
        vl = x[:, 1, :]  # [batch_size, seq_length]

        # Embed sequences
        vh_embedded = self.embedding(vh).transpose(0, 1)  # [seq_length, batch_size, d_model]
        vl_embedded = self.embedding(vl).transpose(0, 1)  # [seq_length, batch_size, d_model]

        # Pass through transformer
        vh_encoded = self.transformer(vh_embedded)  # [seq_length, batch_size, d_model]
        vl_encoded = self.transformer(vl_embedded)  # [seq_length, batch_size, d_model]

        # Pool sequence dimension
        vh_pooled = vh_encoded.mean(dim=0)  # [batch_size, d_model]
        vl_pooled = vl_encoded.mean(dim=0)  # [batch_size, d_model]

        # Concatenate VH and VL features
        combined = torch.cat([vh_pooled, vl_pooled], dim=1)

        # Final MLP layers
        x = F.relu(self.fc1(combined))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=1e-4):
    """Training loop with validation"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for sequences, targets in train_loader:
            sequences = sequences.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for sequences, targets in val_loader:
                sequences = sequences.to(device)
                targets = targets.to(device)

                outputs = model(sequences)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        # Update learning rate
        scheduler.step(val_loss)

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')


In [68]:
def main():
    # Create dataset
    dataset = AntibodyDataset('../data/merged_antibody_data.csv')

    # Split dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=1
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1
    )

    # Initialize model
    model = AntibodyTransformer(
        vocab_size=20,
        d_model=128,
        nhead=8,
        num_layers=3,
        dropout=0.1
    )

    # Train model
    train_model(model, train_loader, val_loader)


In [69]:
main()

/opt/conda/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/opt/conda/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10
Training Loss: 4770.9366
Validation Loss: 4800.4889
Epoch 2/10
Training Loss: 4752.8362
Validation Loss: 4783.6036
Epoch 3/10
Training Loss: 4736.4857
Validation Loss: 4767.1946
Epoch 4/10
Training Loss: 4718.8683
Validation Loss: 4750.1119
Epoch 5/10
Training Loss: 4712.3301
Validation Loss: 4733.1857
Epoch 6/10
Training Loss: 4685.8134
Validation Loss: 4714.5780
Epoch 7/10
Training Loss: 4670.3368
Validation Loss: 4694.2718
Epoch 8/10
Training Loss: 4645.7187
Validation Loss: 4672.0086
Epoch 9/10
Training Loss: 4632.4071
Validation Loss: 4648.1714
Epoch 10/10
Training Loss: 4606.8376
Validation Loss: 4622.3900


### Exercise Tasks

1.  Data Loading and Assessment
    -   Load all three data files
    -   Run initial quality assessment
    -   Identify key data issues
2.  Data Cleaning
    -   Implement temperature standardization
    -   Clean binding affinity values
    -   Handle missing values
    -   Standardize units
3.  Data Integration
    -   Merge datasets
    -   Handle duplicate measurements
    -   Create final feature matrix
4.  ML Model Development
    -   Create feature matrix
    -   Define target variable
    -   Train simple model
    -   Evaluate results

### Bonus Challenges

1.  Advanced Data Cleaning
    -   Implement outlier detection
    -   Add data validation rules
    -   Create data quality reports
2.  Feature Engineering
    -   Create derived features
    -   Implement domain-specific transformations
    -   Add sequence-based features
3.  Model Improvements
    -   Implement cross-validation
    -   Try different ML algorithms
    -   Add uncertainty quantification

### Tips for Success

1.  Data Quality
    -   Always plot your data
    -   Check for outliers
    -   Validate units
    -   Document assumptions
2.  Integration
    -   Verify sample IDs
    -   Check for duplicates
    -   Validate merged data
3.  ML Development
    -   Start simple
    -   Test assumptions
    -   Validate results
